In [10]:
all_operator = {
    "ts1": [
        "days_from_last_change",
        "hump",
        "ts_backfill",
    ],
    "ts2": [
        "last_diff_value",
        "ts_arg_max",
        "ts_arg_min",
        "ts_av_diff",
        "last_diff_value",
        "last_diff_value",
        "ts_count_nans",
        "ts_decay_linear",
        "ts_rank",
        "ts_quantile",
        "ts_product",
        "ts_mean",
        "ts_delta",
        "ts_delay",
        "ts_std_dev",
        "ts_scale",
        "ts_zscore",
        "ts_sum",
    ],
    "ts3": [
        "kth_element"
        "ts_corr",
        "ts_covariance",
        "ts_regression",
    ],
    "cs": [
        "normalize",
        "quantile",
        "rank",
        "zscore",
        "winsorize",
        "scale",
    ],
    "vector":[
        "vec_sum",
        "vec_sum",
    ],
    "group2":[
        "group_zscore",
        "group_scale",
        "group_rank",
        "group_neutralize",
    ],
    "group3":[
        "group_mean",
        "group_backfill",
    ],
}

In [11]:
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from os.path import expanduser
from requests.auth import HTTPBasicAuth
from time import sleep
import pandas as pd
from typing import Dict, List
from time import sleep
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# 文件 handler
file_handler = logging.FileHandler("/root/brain/brain_simulate.log", mode="w")
file_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
logger.addHandler(file_handler)

# 控制台 handler
console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
logger.addHandler(console_handler)

class BrainClient:
    
    def __init__(self, credential_file: str):
        self._simulation_retry = 3
        self._pool = ThreadPoolExecutor(max_workers=1)
        self._simulate_type = 'REGULAR'
        self._simulate_settings = {
            'instrumentType': 'EQUITY',
            'region': 'USA',
            'universe': 'TOP3000',
            'delay': 1,
            'decay': 0,
            'neutralization': 'INDUSTRY',
            'truncation': 0.08,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'OFF',
            'language': 'FASTEXPR',
            'visualization': False
        }
        with open(expanduser(credential_file)) as f:
            credentials = json.load(f)
            self._username, self._password = credentials["username"], credentials["password"]
        self._login()

    def _login(self):
        self._sess = requests.Session()
        self._sess.auth = HTTPBasicAuth(self._username, self._password)
        response = self._sess.post("https://api.worldquantbrain.com/authentication")
        logging.info(f"平台登录状态码：{response.status_code}，平台登录返回信息：{response.json()}")

    def get_datafields(self, searchScope, dataset_id:str, search:str):
        instrument_type = searchScope['instrumentType']
        region = searchScope['region']
        delay = searchScope['delay']
        universe = searchScope['universe']
        if len(search) == 0:
            url_template = 'https://api.worldquantbrain.com/data-fields?' +\
                f'&instrumentType={instrument_type}' +\
                f'region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50&offset={x}'
            count = self._sess.get(url_template.format(x=0)).json()['count']
        else:
            url_template = 'https://api.worldquantbrain.com/data-fields?' +\
                f'&instrumentType={instrument_type}' +\
                f'region={region}&delay={str(delay)}&universe={universe}&search={search}&limit=50&offset={x}'
            count = 100
        datafield_list = []
        for x in range(0,count, 50):
            datafields = self._sess.get(url_template.format(x=x))
            datafield_list.append(datafields.json()["results"])
        datafield_list_flat = [item for sublist in datafield_list for item in sublist]
        datafields_df = pd.DataFrame(datafield_list_flat)
        return datafields_df

    def generate_alpha(self, alpha_template:str, values_list: List[Dict[str, str]])->str:
        return [alpha_template.format_map(values) for values in values_list]

    def _simulate(self, alpha: str):
        simulation_data = {
            'type': self._simulate_type,
            'settings': self._simulate_settings,
            'regular': alpha
        }
        for _ in range(self._simulation_retry):
            sim_resp = self._sess.post(
                'https://api.worldquantbrain.com/simulations',
                json=simulation_data
            )
            if 'Location' not in sim_resp.headers:
                logging.error(f"回测提交失败，alpha: {alpha}, response: {sim_resp.json()}")
                if sim_resp.json()['detail'] == 'Incorrect authentication credentials.':
                    self._login()
            else:
                logging.info(f"回测提交成功，alpha: {alpha}")
        sim_progress_url = sim_resp.headers['Location']
        
        while True:
            sim_progress_resp = self._sess.get(sim_progress_url)
            retry_after_sec = float(sim_progress_resp.headers.get('Retry-After', 0))
            if retry_after_sec == 0:
                break
            sleep(retry_after_sec)
        
        alpha_id = sim_progress_resp.json()['alpha'] 
        logging.info(f"回测运行完成，alpha_id:{alpha_id}，alpha: {alpha}")

    def batch_simulate(self, alpha_list):
        logging.info(f'批量回测的alpha列表为：{alpha_list}')
        self._fetures = [self._pool.submit(self._simulate, (alpha)) for alpha in alpha_list]

    def wait_task_done(self):
        self._pool.shutdown(wait=True)

    

In [12]:
client = BrainClient(credential_file='/root/brain/cerdential')

2025-08-28 17:21:41,487 [INFO] 平台登录状态码：201，平台登录返回信息：{'user': {'id': 'CS62089'}, 'token': {'expiry': 14400.0}, 'permissions': ['TUTORIAL', 'WORKDAY']}
2025-08-28 17:21:41,487 [INFO] 平台登录状态码：201，平台登录返回信息：{'user': {'id': 'CS62089'}, 'token': {'expiry': 14400.0}, 'permissions': ['TUTORIAL', 'WORKDAY']}
2025-08-28 17:21:41,487 [INFO] 平台登录状态码：201，平台登录返回信息：{'user': {'id': 'CS62089'}, 'token': {'expiry': 14400.0}, 'permissions': ['TUTORIAL', 'WORKDAY']}


In [ ]:
field_map_list = []
with open('/root/brain/field_list', 'r') as f:
    for line in f.readlines():
        line = line.strip()
        field_list = line.split('/')
        field_map_list.append(
            {
                "field1": field_list[0],
                "field2": field_list[1],
            }
        )
for op in all_operator['ts2']:
    alpha_template = op+"(- ({field1} - {field2}) / ({field1} + {field2}) , 500)"
    alpha_list = client.generate_alpha(alpha_template=alpha_template, values_list=field_map_list)
    print(alpha_list)
    client.batch_simulate(alpha_list)
    client.wait_task_done()



2025-08-28 17:21:41,504 [INFO] 批量回测的alpha列表为：['last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500)', 'last_diff_value(- (anl4_bvps_low  -  fnd6_newqv1300_recdq) / (anl4_bvps_low  +  fnd6_newqv1300_recdq) , 500)', 'last_diff_value(- (anl4_bvps_low  -  fnd6_recd) / (anl4_bvps_low  +  fnd6_recd) , 500)', 'last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500)', 'last_diff_value(- (anl4_bvps_low  -  news_open) / (anl4_bvps_low  +  news_open) , 500)', 'last_diff_value(- (anl4_bvps_low  -  opt6_slcxp) / (anl4_bvps_low  +  opt6_slcxp) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_cfo_low) / (anl4_capex_high  +  anl4_cfo_low) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_cfo_median) / (anl4_capex_high  +  anl4_cfo_median) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_ebit_low) / (anl4_capex_high  +  anl4_ebit_low) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_ebit_median) / (anl4_cape

['last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500)', 'last_diff_value(- (anl4_bvps_low  -  fnd6_newqv1300_recdq) / (anl4_bvps_low  +  fnd6_newqv1300_recdq) , 500)', 'last_diff_value(- (anl4_bvps_low  -  fnd6_recd) / (anl4_bvps_low  +  fnd6_recd) , 500)', 'last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500)', 'last_diff_value(- (anl4_bvps_low  -  news_open) / (anl4_bvps_low  +  news_open) , 500)', 'last_diff_value(- (anl4_bvps_low  -  opt6_slcxp) / (anl4_bvps_low  +  opt6_slcxp) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_cfo_low) / (anl4_capex_high  +  anl4_cfo_low) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_cfo_median) / (anl4_capex_high  +  anl4_cfo_median) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_ebit_low) / (anl4_capex_high  +  anl4_ebit_low) , 500)', 'last_diff_value(- (anl4_capex_high  -  anl4_ebit_median) / (anl4_capex_high  +  anl4_ebit_median) , 500)', 'last_d

2025-08-28 17:21:42,277 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:42,277 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:42,277 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:42,826 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:42,826 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_high  -  fnd6_recd) / (anl4_bvps_high  +  fnd6_recd) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:2

KeyboardInterrupt: 

2025-08-28 17:21:50,172 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:50,172 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:50,172 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:50,951 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500), response: {'detail': 'CONCURRENT_SIMULATION_LIMIT_EXCEEDED'}
2025-08-28 17:21:50,951 [ERROR] 回测提交失败，alpha: last_diff_value(- (anl4_bvps_low  -  news_eod_close) / (anl4_bvps_low  +  news_eod_close) , 500), response: {'detail': 'CONCURRENT_SIM